In [184]:
import pandas as pd 
df = pd.read_pickle(f"/Users/oscarperezarruti/Documents/Documentos/Repositorios/trabajo-fin-grado-23/src/temp/testings/AAPL.pkl")


# Establecer 'date' como índice
df.set_index('date', inplace=True)

# Filtrar para quedarse con la primera ocurrencia de cada día
df = df.groupby(df.index.date).first()
print(df.columns)


Index(['open', 'high', 'low', 'close', 'volume', 'SMA_200', 'SMA_50', 'RSI_14',
       'beta', 'lastReport', 'reportPriceMovement', 'betaSignal',
       '200smaSignal', '50smaSignal', 'reportSignal', 'minimunSignal',
       'superMinimunSignal', 'rsiSignal', 'minimunSignalPlusRsi'],
      dtype='object')


In [185]:
# Función para verificar si la fecha es lunes o viernes
def is_monday_or_friday(date):
    if date.weekday() == 0:  # 0 representa el lunes
        return 'Lunes'
    elif date.weekday() == 4:  # 4 representa el viernes
        return 'Viernes'
    else:
        return None  # En caso de que no sea lunes ni viernes

# Asegurarse de que el índice es de tipo datetime
df.index = pd.to_datetime(df.index)

# Aplicar la función al índice y crear una nueva columna 'day_type'
df['day_type'] = df.index.map(is_monday_or_friday)

print(df["day_type"])

num_mondays = len(df[df['day_type'] == 'Lunes'])
num_fridays = len(df[df['day_type'] == 'Viernes'])

# Mostrar el resultado
print(f'Días que son Lunes: {num_mondays}')
print(f'Días que son Viernes: {num_fridays}')

2021-01-04      Lunes
2021-01-05       None
2021-01-06       None
2021-01-07       None
2021-01-08    Viernes
               ...   
2023-11-27      Lunes
2023-11-28       None
2023-11-29       None
2023-11-30       None
2023-12-01    Viernes
Name: day_type, Length: 734, dtype: object
Días que son Lunes: 134
Días que son Viernes: 148


In [186]:
df.dropna(subset=['day_type'], inplace=True)
print(df["day_type"])

2021-01-04      Lunes
2021-01-08    Viernes
2021-01-11      Lunes
2021-01-15    Viernes
2021-01-22    Viernes
               ...   
2023-11-17    Viernes
2023-11-20      Lunes
2023-11-24    Viernes
2023-11-27      Lunes
2023-12-01    Viernes
Name: day_type, Length: 282, dtype: object


In [187]:

dates_to_remove = []

# Iterar sobre el DataFrame
for index, row in df.iterrows():
    if row['day_type'] == 'Lunes':
        # Buscar la siguiente fecha que sea viernes, considerando también si hay otro lunes en medio
        for next_index, next_row in df[(df.index > index)].iterrows():
            if next_row['day_type'] == 'Lunes':
                # Si se encuentra otro lunes antes, eliminar solo este lunes
                dates_to_remove.append(index)
                break
            elif next_row['day_type'] == 'Viernes':
                # Si se encuentra un viernes, verificar la diferencia de días
                if (next_index - index).days != 4:
                    # Si la diferencia de días no es cuatro, eliminar ambos
                    dates_to_remove.extend([index, next_index])
                break

# Eliminar las fechas marcadas del DataFrame
df = df.drop(dates_to_remove)

# Mostrar el DataFrame resultante
print(df.columns)

Index(['open', 'high', 'low', 'close', 'volume', 'SMA_200', 'SMA_50', 'RSI_14',
       'beta', 'lastReport', 'reportPriceMovement', 'betaSignal',
       '200smaSignal', '50smaSignal', 'reportSignal', 'minimunSignal',
       'superMinimunSignal', 'rsiSignal', 'minimunSignalPlusRsi', 'day_type'],
      dtype='object')


In [188]:

# Iterar sobre las filas de 'Viernes'
for index, row in df[df['day_type'] == 'Lunes'].iterrows():
    # Calcular la fecha del lunes asociado
    friday_date = index + pd.Timedelta(days=4)

    # Verificar si el lunes existe en el DataFrame
    if friday_date in df.index:
        friday_row = df.loc[friday_date]

        growth = friday_row['close'] - row['close']


        # Asignar el valor de crecimiento a la fila de viernes
        df.at[index, 'growth'] = growth

# Mostrar el DataFrame resultante
print(df.columns)
print(df[['day_type', 'growth']])

Index(['open', 'high', 'low', 'close', 'volume', 'SMA_200', 'SMA_50', 'RSI_14',
       'beta', 'lastReport', 'reportPriceMovement', 'betaSignal',
       '200smaSignal', '50smaSignal', 'reportSignal', 'minimunSignal',
       'superMinimunSignal', 'rsiSignal', 'minimunSignalPlusRsi', 'day_type',
       'growth'],
      dtype='object')
           day_type  growth
2021-01-04    Lunes  -1.808
2021-01-08  Viernes     NaN
2021-01-11    Lunes  -2.231
2021-01-15  Viernes     NaN
2021-01-22  Viernes     NaN
...             ...     ...
2023-11-17  Viernes     NaN
2023-11-20    Lunes   1.850
2023-11-24  Viernes     NaN
2023-11-27    Lunes   0.700
2023-12-01  Viernes     NaN

[278 rows x 2 columns]


In [189]:
# Filtrar el DataFrame original para obtener solo las filas con 'day_type' igual a 'Lunes'
df_lunes = df[df['day_type'] == 'Lunes']


new_df_lunes = df_lunes[['growth', 'minimunSignal']].copy()


# Mostrar el nuevo DataFrame
print(new_df_lunes)


            growth  minimunSignal
2021-01-04  -1.808          False
2021-01-11  -2.231          False
2021-01-25  -6.388          False
2021-02-01   3.375          False
2021-02-08  -2.598          False
...            ...            ...
2023-10-30   3.260          False
2023-11-06   5.343           True
2023-11-13   4.770           True
2023-11-20   1.850           True
2023-11-27   0.700          False

[130 rows x 2 columns]


In [190]:
# Sumar todas las veces que la columna 'minimunSignal' es True
total_true = new_df_lunes['minimunSignal'].sum()

# Mostrar el resultado
print("Número de veces que 'minimunSignal' es True:", total_true)


Número de veces que 'minimunSignal' es True: 35


In [191]:
# Filtrar para obtener solo las filas donde 'minimunSignal' es True
df_true = new_df_lunes[new_df_lunes['minimunSignal'] == True]

# Mostrar el nuevo DataFrame
print(df_true)


            growth  minimunSignal
2021-04-05   6.702           True
2021-04-12   1.565           True
2021-04-19  -2.165           True
2021-04-26  -1.319           True
2021-05-10  -3.825           True
2021-05-17   0.562           True
2021-05-24   0.158           True
2021-06-07   1.084           True
2021-06-14   4.190           True
2021-07-12   3.105           True
2021-07-26  -3.007           True
2021-08-02   0.315           True
2021-08-16  -2.695           True
2021-08-23  -0.474           True
2021-08-30   5.331           True
2021-09-27  -6.507           True
2023-04-17   1.056           True
2023-05-01   0.687           True
2023-05-08   0.846           True
2023-05-15   2.264           True
2023-05-22  -2.423           True
2023-06-05  -1.486           True
2023-06-12   4.319           True
2023-06-26   3.959           True
2023-07-03  -2.613           True
2023-07-17   2.185           True
2023-07-31  -7.271           True
2023-08-14  -4.853           True
2023-08-28   9

In [192]:
# Sumar todas las veces que la columna 'minimunSignal' es True
total_growth = new_df_lunes['growth'].sum()

# Mostrar el resultado
print("Crecimiento total:", round(total_growth,2),"$")


Crecimiento total: 50.99 $
